# Dynamic Programming

The term dynamic programming (DP) refers to a collection of algorithms that can be used to compute optimal policies given a perfect model of the environment as a Markov decision process (MDP). Classical DP algorithms are of limited utility in reinforcement learning both because of their assumption of a perfect model and because of their great computational expense, but they are still important theoretically. DP provides an essential foundation for the understanding of the methods presented in the rest of this book. In fact, all of these methods can be viewed as attempts to achieve much the same effect as DP, only with less computation and without assuming a perfect model of the environment.\
术语动态规划（dynamic programming，DP）指的是一组算法，可以用来计算最优策略给出一个完美的环境模型作为马尔科夫决策过程（MDP）。经典的DP算法由于其假设是一个完美的模型，且计算量大，在强化学习方面的应用有限，但在理论上仍然很重要。DP为理解在这本书的其余部分中提出的方法提供了必要的基础。事实上，所有这些方法都可以被视为试图达到与DP相同的效果，只是需要更少的计算，而且没有假设一个完美的环境模型。

## Policy Evaluation (Prediction)

First we consider how to compute the state-value function $v_\pi$ for an arbitrary policy $\pi$. This is called **policy evaluation** in the DP literature. We also refer to it as the **prediction problem**. Recall from Chapter 3 that, for all $s \in \mathcal{S}$,\
首先考虑如何计算任意策略$\pi$的状态值函数$v_\pi$。这在DP文献中被称为__策略评估（policy evaluation）__。我们也把它称为__预测问题（prediction problem）__。回想一下第3章，对于所有$s \in \mathcal{S}$，
$$
\begin{align}
v_{\pi}(s) 
&= \mathbb{E}_\pi [G_t | S_t=s] \\
&= \mathbb{E}_\pi [R_{t+1} + \gamma G_{t+1} | S_t=s], \qquad \tag{from (3.9)} \\
&= \mathbb{E}_\pi [R_{t+1} + \gamma v_\pi(S_{t+1}) | S_t=s] \tag{4.3} \\
&= \sum_a \pi(a|s) \sum_{s',r} p(s',r|s,a) [r + \gamma v_\pi(s')] \tag{4.4}
\end{align}
$$

A complete in-place version of iterative policy evaluation is shown in pseudocode in the box below. Note how it handles termination. Formally, iterative policy evaluation converges only in the limit, but in practice it must be halted short of this. The pseudocode tests the quantity $\max_{s \in \mathcal{S}} |v_{k+1}(s)−v_k(s)|$ after each sweep and stops when it is sufficiently small.\
下面方框中的伪代码显示了迭代策略评估的完整in-place版本。注意它是如何处理终止的。形式上，迭代策略评估只在极限上收敛，但在实践中必须停止这种收敛。伪代码在每次扫描后测试量$\max_{s \in \mathcal{S}} |v_{k+1}(s)−v_k(s)|$，当它足够小时停止。

__<font color = red>Algorithm</font> Iterative Policy Evaluation, for estimating $V = v_{\pi}$__




**<font color = green>Example 4.1</font> $4 \times 4$ Gridworld**\
Consider the $4 \times 4$ gridworld shown below.

<img src = "RL Example 4_1.png">

The nonterminal states are $S = \{1, 2, \cdots, 14\}$. There are four actions possible in each state, $\mathcal{A} = \{up, down, right, left\}$, which deterministically cause the corresponding state transitions, except that actions that would take the agent off the grid in fact leave the state unchanged. Thus, for instance, $p(6,-1|5,right) = 1$, $p(7,-1|7,right) = 1$, and $p(10,r|5,right) = 0$ for all $r \in \mathcal{R}$. This is an undiscounted, episodic task. The reward is -1 on all transitions until the terminal state is reached. The terminal state is shaded in the figure (although it is shown in two places, it is formally one state). The expected reward function is thus $r(s, a, s') = -1$ for all states $s$, $s'$ and actions $a$. Suppose the agent follows the equiprobable random policy (all actions equally likely). The left side of Figure 4.1 shows the sequence of value functions $\{v_k\}$ computed by iterative policy evaluation. The final estimate is in fact $v_\pi$, which in this case gives for each state the negation of the expected number of steps from that state until termination.\
非终止状态是$S = \{1, 2, \cdots, 14\}$。每个状态中可能有4个操作，$\mathcal{A} = \{up, down, right, left\}$，它们确定地导致相应的状态转换，除非将代理从网格中移除的操作实际上保持状态不变。因此，例如，对于所有的$r \in \mathcal{R}$，有$p(6,-1|5,right) = 1$，$p(7,-1|7,right) = 1$和$p(10,r|5,right) = 0$。这是一个不打折扣的、断断续续的任务。在到达终止状态之前，所有转换的奖励都是-1。terminal状态在图中有阴影（尽管它在两个地方显示，但形式上是一种状态）。因此，对于所有的状态$s$、状态$s'$和动作$a$，期望的奖励函数为$r(s, a, s') = -1$。假设代理遵循等可能的随机策略（所有动作都是等可能的）。图4.1左侧显示了通过迭代策略计算得到的值函数$\{v_k\}$的序列。最终的估计实际上是$v_\pi$，在这种情况下，它为每个状态给出了从该状态到终止的预期步骤数的否定。

<img src = "RL Figure 4_1.png">

**Figure 4.1:**\
Convergence of iterative policy evaluation on a small gridworld. The left column is the sequence of approximations of the state-value function for the random policy (all actions equally likely). The right column is the sequence of greedy policies corresponding to the value function estimates (arrows are shown for all actions achieving the maximum, and the numbers shown are rounded to two significant digits). The last policy is guaranteed only to be an improvement over the random policy, but in this case it, and all policies after the third iteration, are optimal.\
小网格世界迭代策略评估的收敛性。左列是随机策略(所有动作都是等可能的)的状态值函数的近似序列。右列是与值函数估计相对应的贪婪策略序列(箭头显示所有达到最大值的操作，所显示的数字四舍六入到两个有效数字)。最后一个策略被保证只是对随机策略的改进，但在这种情况下，它和第三次迭代后的所有策略都是最优策略。

## Policy Improvement

## Policy Iteration

## Value Iteration

One drawback to policy iteration is that each of its iterations involves policy evaluation, which may itself be a protracted iterative computation requiring multiple sweeps through the state set. If policy evaluation is done iteratively, then convergence exactly to $v_\pi$ occurs only in the limit. Must we wait for exact convergence, or can we stop short of that? The example in Figure 4.1 certainly suggests that it may be possible to truncate policy evaluation. In that example, policy evaluation iterations beyond the first three have no effect on the corresponding greedy policy.\
策略迭代的一个缺点是，它的每个迭代都涉及到策略评估，而策略评估本身可能是一种拖长的迭代计算，需要对状态集进行多次扫描。如果策略计算是迭代的，那么收敛到$v_\pi$仅在极限情况下发生。我们是必须等待确切的收敛，还是能在此之前停止？图4.1中的例子显然表明可以截断策略评估。在这个例子中，超过前三个的策略评估迭代对相应的贪婪策略没有影响。

In fact, the policy evaluation step of policy iteration can be truncated in several ways without losing the convergence guarantees of policy iteration. One important special case is when policy evaluation is stopped after just one sweep (one update of each state). This algorithm is called __value iteration__. It can be written as a particularly simple update operation that combines the policy improvement and truncated policy evaluation steps:\
事实上，在不失去策略迭代收敛性保证的情况下，可以用多种方式截断策略迭代的策略评估步骤。一个重要的特殊情况是在一次扫描(每个状态更新一次)之后停止策略评估。这种算法称为__值迭代__。它可以写成一个特别简单的更新操作，它结合了策略改进和截断的策略评估步骤：

Finally, let us consider how value iteration terminates. Like policy evaluation, value iteration formally requires an infinite number of iterations to converge exactly to $v_*$. In practice, we stop once the value function changes by only a small amount in a sweep. The box below shows a complete algorithm with this kind of termination condition.\
最后，让我们考虑一下值迭代是如何结束的。与策略评估一样，值迭代正式要求无限次迭代才能精确收敛到$v_*$。在实际操作中，一旦值函数在一次扫描中只发生了很小的变化，我们就停止。下面的框显示了具有这种终止条件的完整算法。